In [1]:
pip install pandas numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.Defaulting to user installation because normal site-packages is not writeable




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\himan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd

# Load datasets
movies = pd.read_csv(r"C:\Users\himan\Downloads\movies.csv")  # Contains movieId, title, genres
ratings = pd.read_csv(r"C:\Users\himan\Downloads\ratings.csv")  # Contains userId, movieId, rating

# Display the first few rows
print(movies.head())
print(ratings.head())


   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510


In [7]:
# Limit the dataset to the first 10,000 movies
movies_subset = movies.head(10000)
count_matrix = count_vectorizer.fit_transform(movies_subset['genres'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)


In [8]:
from sklearn.metrics.pairwise import linear_kernel

# Use linear_kernel instead of cosine_similarity
cosine_sim = linear_kernel(count_matrix, count_matrix)


In [9]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Load a smaller subset
movies_subset = movies.head(10000)

# Apply CountVectorizer and linear_kernel
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(movies_subset['genres'])
cosine_sim = linear_kernel(count_matrix, count_matrix)

# Proceed with recommendation function using smaller cosine_sim matrix
indices = pd.Series(movies_subset.index, index=movies_subset['title']).drop_duplicates()

def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    return movies_subset['title'].iloc[movie_indices]

print(get_recommendations('Toy Story (1995)'))


661                                   Space Jam (1996)
2203                                       Antz (1998)
2895                   Who Framed Roger Rabbit? (1988)
3021                                Toy Story 2 (1999)
3653    Adventures of Rocky and Bullwinkle, The (2000)
Name: title, dtype: object


In [11]:
print("Max userId:", ratings['userId'].max())
print("Max movieId:", ratings['movieId'].max())


Max userId: 162541
Max movieId: 209171


In [12]:
ratings = ratings[(ratings['userId'] <= ratings['userId'].max()) & (ratings['movieId'] <= ratings['movieId'].max())]

In [13]:
ratings = ratings[(ratings['userId'] <= ratings['userId'].max()) & (ratings['movieId'] <= ratings['movieId'].max())]


In [16]:
# Select a subset of users and movies to reduce memory usage
ratings_subset = ratings[(ratings['userId'] <= 1000) & (ratings['movieId'] <= 500)]
user_movie_matrix = ratings_subset.pivot(index='userId', columns='movieId', values='rating').fillna(0)


In [17]:
# Optimize data types to save memory
ratings['userId'] = ratings['userId'].astype('int32')
ratings['movieId'] = ratings['movieId'].astype('int32')
ratings['rating'] = ratings['rating'].astype('float32')


In [18]:
from scipy.sparse import csr_matrix

# Convert the user_movie_matrix to a sparse matrix format
user_movie_matrix_sparse = csr_matrix(user_movie_matrix.values)


In [20]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

# Optimize data types
ratings['userId'] = ratings['userId'].astype('int32')
ratings['movieId'] = ratings['movieId'].astype('int32')
ratings['rating'] = ratings['rating'].astype('float32')

# Use a smaller subset to reduce memory usage
ratings_subset = ratings[(ratings['userId'] <= 1000) & (ratings['movieId'] <= 500)]
user_movie_matrix = ratings_subset.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Convert to sparse matrix
user_movie_matrix_sparse = csr_matrix(user_movie_matrix.values)

# Apply SVD on sparse matrix
svd = TruncatedSVD(n_components=12)
matrix = svd.fit_transform(user_movie_matrix_sparse)

# Display the shape of the transformed matrix
print(matrix.shape)


(968, 12)


In [21]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Calculate cosine similarity between users
user_similarity = cosine_similarity(matrix)

# Convert similarity matrix to DataFrame for easy lookup
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

def recommend_movies(user_id, num_recommendations=5):
    # Get user similarity scores for the given user
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).iloc[1:num_recommendations+1]
    
    # Get the movies rated by similar users
    similar_user_ids = similar_users.index
    similar_user_ratings = user_movie_matrix.loc[similar_user_ids]
    
    # Average the ratings for each movie, weighted by user similarity scores
    weighted_ratings = similar_user_ratings.T.dot(similar_users) / similar_users.sum()
    
    # Sort movies based on weighted ratings and recommend the top ones
    recommended_movies = weighted_ratings.sort_values(ascending=False).head(num_recommendations)
    
    return recommended_movies.index.tolist()

# Example: Recommend movies for user with ID 1
recommended_movie_ids = recommend_movies(user_id=1, num_recommendations=5)
print("Recommended movie IDs:", recommended_movie_ids)


Recommended movie IDs: [296, 32, 235, 154, 111]


In [22]:
# Assuming 'movies' DataFrame has been loaded
recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]
print(recommended_movies[['movieId', 'title']])


     movieId                                      title
31        32  Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
109      111                         Taxi Driver (1976)
152      154   Beauty of the Day (Belle de jour) (1967)
232      235                             Ed Wood (1994)
292      296                        Pulp Fiction (1994)


In [4]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

# Load datasets (replace "path/to" with the actual file path)
movies = pd.read_csv(r"C:\Users\himan\Downloads\movies.csv")  # Contains movieId, title, genres
ratings = pd.read_csv(r"C:\Users\himan\Downloads\ratings.csv")  

# Optimize data types
ratings['userId'] = ratings['userId'].astype('int32')
ratings['movieId'] = ratings['movieId'].astype('int32')
ratings['rating'] = ratings['rating'].astype('float32')

# Use a smaller subset to reduce memory usage (adjust as necessary)
ratings_subset = ratings[(ratings['userId'] <= 1000) & (ratings['movieId'] <= 500)]
user_movie_matrix = ratings_subset.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Convert to sparse matrix
user_movie_matrix_sparse = csr_matrix(user_movie_matrix.values)

# Apply SVD on sparse matrix
svd = TruncatedSVD(n_components=12)
matrix = svd.fit_transform(user_movie_matrix_sparse)

# Function to recommend movies
from sklearn.metrics.pairwise import cosine_similarity
def recommend_movies(user_id, num_recommendations=5):
    user_similarity = cosine_similarity(matrix)
    user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).iloc[1:num_recommendations+1]
    similar_user_ids = similar_users.index
    similar_user_ratings = user_movie_matrix.loc[similar_user_ids]
    weighted_ratings = similar_user_ratings.T.dot(similar_users) / similar_users.sum()
    recommended_movies = weighted_ratings.sort_values(ascending=False).head(num_recommendations)
    return recommended_movies.index.tolist()

# CLI for Recommendations
def user_menu():
    while True:
        print("\nMovie Recommendation System")
        print("1. Get Recommendations for a User")
        print("2. Search for a Movie and Get Recommendations")
        print("3. Exit")
        choice = input("Enter your choice (1/2/3): ")

        if choice == '1':
            try:
                user_id = int(input("Enter user ID: "))
                recommended_movie_ids = recommend_movies(user_id=user_id, num_recommendations=5)
                recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]
                print("\nRecommended Movies:")
                print(recommended_movies[['movieId', 'title']])
            except KeyError:
                print("User ID not found. Please try again.")

        elif choice == '2':
            search_title = input("Enter a movie title to find similar recommendations: ").lower()
            movie_matches = movies[movies['title'].str.lower().str.contains(search_title, regex=False)]

            if movie_matches.empty:
                print("No movies found with that title.")
            else:
                print("Movies found:")
                for idx, row in movie_matches.iterrows():
                    print(f"{row['movieId']}: {row['title']}")
                try:
                    selected_movie_id = int(input("Enter the movie ID from the list above: "))
                    similar_movies = recommend_movies(selected_movie_id, num_recommendations=5)
                    print("\nSimilar Movies:")
                    print(similar_movies[['movieId', 'title']])
                except (ValueError, KeyError):
                    print("Invalid movie ID. Please try again.")

        elif choice == '3':
            print("Exiting the system.")
            break
        else:
            print("Invalid choice. Please try again.")

# Run the CLI
user_menu()



Movie Recommendation System
1. Get Recommendations for a User
2. Search for a Movie and Get Recommendations
3. Exit

Recommended Movies:
     movieId                                      title
167      169    Free Willy 2: The Adventure Home (1995)
257      260  Star Wars: Episode IV - A New Hope (1977)
314      318           Shawshank Redemption, The (1994)
324      329              Star Trek: Generations (1994)
357      362                    Jungle Book, The (1994)

Movie Recommendation System
1. Get Recommendations for a User
2. Search for a Movie and Get Recommendations
3. Exit
Invalid choice. Please try again.

Movie Recommendation System
1. Get Recommendations for a User
2. Search for a Movie and Get Recommendations
3. Exit
Invalid choice. Please try again.

Movie Recommendation System
1. Get Recommendations for a User
2. Search for a Movie and Get Recommendations
3. Exit
Invalid choice. Please try again.

Movie Recommendation System
1. Get Recommendations for a User
2. Search 